In [14]:
import cv2
from sklearn.metrics import accuracy_score # Accuracy metrics 
import pickle 
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import warnings
import mediapipe as mp # Import mediapipe
import numpy as np
import pandas as pd
# Suppress specific sklearn warnings
warnings.filterwarnings("ignore", category=UserWarning, module="sklearn")
mp_holistic = mp.solutions.holistic # Mediapipe Solutions


In [12]:
def feature_detect_setup():

    filename = '/Users/kailashambwani/Dropbox/PitchPerfect/body_models/face_model_gb.pkl'
    with open(filename, 'rb') as f:
        face_model = pickle.load(f)

    filename = '/Users/kailashambwani/Dropbox/PitchPerfect/body_models/pose_model_rf.pkl'
    with open(filename, 'rb') as f:
        pose_model = pickle.load(f)
        
    return face_model, pose_model

def feature_detect(frame, face_model, pose_model):
    # Recolor Feed
    image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False        

    # Make Detections
    with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:

        results = holistic.process(image)

    try:
        # Extract Pose landmarks
        pose = results.pose_landmarks.landmark
        pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())

        # Extract Face landmarks
        face = results.face_landmarks.landmark
        face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())

#             # Concate rows
#             row = pose_row+face_row

        # Make face Detections
        face_X = pd.DataFrame([face_row])
        face_class = face_model.predict(face_X)[0]
        face_prob = face_model.predict_proba(face_X)[0]
#            print(face_class, face_prob)

        # Make pose Detections
        pose_X = pd.DataFrame([pose_row])
        pose_class = pose_model.predict(pose_X)[0]
        pose_prob = pose_model.predict_proba(pose_X)[0]
#            print(pose_class, pose_prob)

    except:
        pass

    return face_class, pose_class

In [21]:

def extract_features_from_video(video_file, face_model, pose_model):
    """
    Extract features from a video using the body_detect function.
    
    Args:
    - video_file (str): Path to the video file.
    - body_detect (func): Function to detect features in an image frame.
    
    Returns:
    - List[Tuple[float, Any]]: List of timestamps and their detected features.
    """
    # List to store the results
    results = []
    curr_features = ('','')
    # Open the video
    cap = cv2.VideoCapture(video_file)
    
    # Check if video opened successfully
    if not cap.isOpened():
        print("Error: Couldn't open video.")
        return results
    
    # Get the frames per second of the video
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    while True:
        # Read the next frame from the video
        ret, frame = cap.read()
        
        # Break the loop if video has ended
        if not ret:
            break
        
        # Get the current position in the video in seconds
        timestamp = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0
        
        # Detect features in the frame
        features = feature_detect(frame, face_model, pose_model)
        if curr_features != features:
            curr_features = features
            print(timestamp, features)
            # Append the timestamp and features to the results list
            results.append((timestamp, features))
    
    # Release the video capture object
    cap.release()
    
    return results

# Example usage:
# def body_detect(frame):
#     # Your detection code here
#     return detected_features

# video_file = 'path_to_video.mp4'
# features_list = extract_features_from_video(video_file, body_detect)
# print(features_list)


In [22]:

if __name__ == '__main__':

# Open the video file
    video_file = '/Users/kailashambwani/Dropbox/PitchPerfect/ray_video.mov'
# cap = cv2.VideoCapture(video_path)

# if not cap.isOpened():
#     print("Error: Couldn't open video.")
# else:
#     while True:
#         # Read the next frame from the video
#         ret, frame = cap.read()
        
#         # If ret is False, then we've reached the end of the video
#         if not ret:
#             break
        
#         # Now, 'frame' contains the current frame and you can process it if needed
#         # For demonstration, let's display the frame
#         cv2.imshow("Frame", frame)
        
#         # Wait for 25ms, and then display the next frame. 
#         # This will play the video at a normal speed.
#         # If you press 'q', it will exit the loop and stop displaying the video.
#         if cv2.waitKey(25) & 0xFF == ord('q'):
#             break

#     # Release the video capture object and close all OpenCV windows
#     cap.release()
#     cv2.destroyAllWindows()

    face_model, pose_model = feature_detect_setup()
    results = extract_features_from_video(video_file, face_model, pose_model)
    print(results)

0.0 ('frown', 'hands pocket')
1.8998100000000002 ('smile', 'hands pocket')
4.09959 ('smile', 'arms crossed')
5.26614 ('smile', 'hands pocket')
5.332800000000001 ('smile', 'arms crossed')
5.432790000000001 ('smile', 'hands pocket')
5.46612 ('smile', 'arms crossed')
5.6661 ('smile', 'hands pocket')
5.732760000000001 ('smile', 'arms crossed')
5.76609 ('smile', 'hands pocket')
5.832750000000001 ('smile', 'arms crossed')
6.332700000000001 ('smile', 'hands pocket')
6.699330000000001 ('smile', 'arms crossed')
6.999300000000001 ('smile', 'hands pocket')
7.03263 ('smile', 'arms crossed')
10.19898 ('smile', 'hands pocket')


KeyboardInterrupt: 

In [16]:
results


NameError: name 'results' is not defined